In [1]:
import pandas as pd
import joblib
import os

os.chdir('../')

In [2]:
import mlflow

experiment_name = "optimizacion"  # Reemplaza <nombre_del_experimento> con el nombre del experimento que deseas buscar

experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is not None:
    experiment_id = experiment.experiment_id
    print("Experiment ID:", experiment_id)
else:
    print("El experimento no existe.")

Experiment ID: 910525732961037228


In [3]:
experiment_id = '910525732961037228'

In [4]:
runs = mlflow.search_runs(experiment_ids=experiment_id,order_by=['metrics.MAPE_test ASC'], max_results=1).iloc[0]

In [5]:
runs

run_id                                      4fb19a084f734574ac15c095b896d401
experiment_id                                             910525732961037228
status                                                              FINISHED
artifact_uri               file:///e:/Importante/Estudios/Maestria/11%20D...
start_time                                  2023-11-20 03:56:38.477000+00:00
end_time                                    2023-11-20 03:58:07.052000+00:00
metrics.MAPE_test                                                    0.21172
metrics.MAPE_train                                                   0.15306
params.max_depth                                                           4
params.learning_rate                                     0.16946215321744781
params.n_estimators                                                      436
tags.mlflow.runName                                          optimizacion_gb
tags.mlflow.user                                                       Santi

In [6]:
params = {
    'max_depth': int(runs['params.max_depth']),
    'n_estimators': int(runs['params.n_estimators']),
    'learning_rate': float(runs['params.learning_rate']),
}

name = {
    'model_name': runs['tags.mlflow.runName']
    }

metrics={
     'MAPE_TEST':runs['metrics.MAPE_test'],
     'MAPE_TRAIN':runs['metrics.MAPE_train']
}

print("Mejor modelo: ",name)
print("Parámetros: ",params)
print("Métricas: ",metrics)

Mejor modelo:  {'model_name': 'optimizacion_gb'}
Parámetros:  {'max_depth': 4, 'n_estimators': 436, 'learning_rate': 0.16946215321744781}
Métricas:  {'MAPE_TEST': 0.21171960423262826, 'MAPE_TRAIN': 0.15305954217401735}


In [7]:
df_train = joblib.load("datos/data_train.joblib")
df_test = joblib.load("datos/data_test.joblib")
data_final=pd.concat([df_train,df_test])

In [8]:
data_final.shape

(12694, 90)

In [9]:
fwiz = joblib.load("modelos/fwiz.joblib")
cat_econder = joblib.load("modelos/cat_econder.joblib")

Imported 0.3.2 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False, skip_xgboost=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



In [13]:
X_t= cat_econder.transform(data_final.drop(columns=['clean_valor_total_avaluo']))

In [14]:
X_selected = fwiz.transform(X_t)
y = data_final['clean_valor_total_avaluo']

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state=123,**params)
reg.fit(X_selected, y)

GradientBoostingRegressor(learning_rate=0.16946215321744781, max_depth=4,
                          n_estimators=436, random_state=123)

In [16]:
import joblib

fwiz = joblib.load("modelos/fwiz.joblib")
cat_econder = joblib.load("modelos/cat_econder.joblib")
vars_dict = joblib.load("modelos/vars_dict.joblib")

In [17]:
from sklearn.pipeline import make_pipeline

pred_pipeline = make_pipeline(cat_econder, fwiz, reg)

In [18]:
joblib.dump(pred_pipeline, "modelo-pkg/model/trained/modelo_avaluos-0.0.1.joblib")

['modelo-pkg/model/trained/modelo_avaluos-0.0.1.joblib']